In [212]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import scipy.io as sio 
import scipy
from scipy import stats
from scipy.stats import norm # normal distribution
from scipy import optimize

import matplotlib
import matplotlib.pyplot as plt

import math
import os
import random

In [213]:
# read paths
directory = os.listdir('../chip02/p655')
directory.sort()
print('../chip02/p655/' + directory[0])

../chip02/p655/180518_ch02v050r02d3_int44_time10000.mat


In [214]:
# read binary data
data = []
for k, v in enumerate(directory):
    data.append(sio.loadmat('../chip02/p655/' + directory[k]))

In [215]:
# prepare lists
fmax_ary = []
dfof_ary = []
wid_data = []
wid_pp = []
rtn_pp = []
rtn_pp_arr = []
df = []
for k, v in enumerate(data):
    fmax_ary.append(np.array([]))
    dfof_ary.append(np.array([]))

In [216]:
# Extract WID and RTN data
for k, v in enumerate(data):
    dfof_ary[k] = pd.DataFrame(columns=['slot_num', 'delta_f'])
    for key, value in data[k].items():
        if key[0] == '_':
            continue
        fmax_ary[k] = np.append(fmax_ary[k], value.max())
        dfof_ary[k].loc[key] = [round(int(key.replace('s', ''))), (value.max() - value.min()) / value.max()]
    wid_data.append((fmax_ary[k] - fmax_ary[k].mean()) / fmax_ary[k].mean() * 100)
    wid_pp.append(stats.probplot(wid_data[k]))
    # wid_pp.append(stats.probplot(fmax_ary[k]))
    dfof_ary[k] = dfof_ary[k].sort_values('delta_f')
    dfof_ary[k]['y'] = stats.probplot(dfof_ary[k]['delta_f'])[0][0].tolist()
    dfof_ary[k]['x'] = stats.probplot(dfof_ary[k]['delta_f'])[0][1].tolist()
#     print(np.corrcoef(wid_data[k], dfof_ary[k]))

# 最小二乗法を用いて近似式を導出

In [217]:
# 最小二乗法を用いて近似式を導出

def fit_func(parameter,x,y):
    a=parameter[0]
    b=parameter[1]
    residual=y-(a*x+b)
    return residual

parameter0=[-6,-4]

least_squares_arr =[]
for k, v in enumerate(data):
    x_arr = np.log(dfof_ary[k]['x'].values)
    y_arr = dfof_ary[k]['y'].values
    result = scipy.optimize.leastsq(fit_func,parameter0,args=(x_arr,y_arr))[0].tolist()
    
    # y = a * log(x) + b を x = 変換
    result.append(1/ result[0])
    result.append(-result[1]/result[0]) 
    
    # 近似直線用の2点    
    result.append(min(dfof_ary[k]['y']))
    result.append(max(dfof_ary[k]['y']))
    
    result.append(math.exp(min(dfof_ary[k]['y'])*result[2] + result[3]))
    result.append(math.exp(max(dfof_ary[k]['y'])*result[2] + result[3]))
    
    least_squares_arr.append(result)
df = pd.DataFrame(least_squares_arr, 
                                  index=['N=140', 'N=280', 'N=560', 'N=1120', 'N=2240'],
                                  columns=['a', 'b', r'$\sigma$', r'$\mu$', 'y_min', 'y_max', 'x_min', 'x_max'])
df

,a,b,$\sigma$,$\mu$,y_min,y_max,x_min,x_max
N=140,3.319282,13.013758,0.301270,-3.920654,-2.905721,2.905721,0.008262,0.047585
N=280,4.034902,17.169270,0.247837,-4.255188,-2.905721,2.905721,0.006906,0.029158
N=560,4.655244,21.432948,0.214811,-4.604043,-2.905721,2.905721,0.005363,0.018688
N=1120,5.885608,29.374513,0.169906,-4.990906,-2.905721,2.905721,0.004150,0.011140
N=2240,6.475380,34.640114,0.154431,-5.349511,-2.905721,2.905721,0.003033,0.007441


In [218]:
fig, ax = plt.subplots()
plt.title("QQ-plot for P = 655")
plt.xscale("log")
font = {'size' : 21}
matplotlib.rc('font', **font)
# x = np.arange(-0.01, 0, 0.001)
# print(x*4)
# print(math.exp(df.iloc[0,0] + df.iloc[0,1]))
# y = math.exp(df.iloc[0,0]*x + df.iloc[0,1])
# print(y)
# plt.plot(x, y)

plt.plot([df.iat[0, 6], df.iat[0, 7]], [df.iat[0, 4], df.iat[0, 5]], color='blue')
plt.plot([df.iat[1, 6], df.iat[1, 7]], [df.iat[1, 4], df.iat[1, 5]], color='red')
plt.plot([df.iat[2, 6], df.iat[2, 7]], [df.iat[2, 4], df.iat[2, 5]], color='green')
plt.plot([df.iat[3, 6], df.iat[3, 7]], [df.iat[3, 4], df.iat[3, 5]], color='orange')
plt.plot([df.iat[4, 6], df.iat[4, 7]], [df.iat[4, 4], df.iat[4, 5]], color='yellow')

dfof_ary[0].plot.scatter(x='x', y='y', s=20, label='N=140', ax=ax, color='blue',edgecolors='blue')
dfof_ary[1].plot.scatter(x='x', y='y', s=20, label='N=280', ax=ax, color='red',edgecolors='red')
dfof_ary[2].plot.scatter(x='x', y='y', s=20, label='N=560', ax=ax, color='green',edgecolors='green')
dfof_ary[3].plot.scatter(x='x', y='y', s=20, label='N=1120', ax=ax, color='orange',edgecolors='orange')
dfof_ary[4].plot.scatter(x='x', y='y', s=20, label='N=2240', ax=ax, color='yellow',edgecolors='yellow', figsize=(20,20))
plt.xlabel(r'$\Delta$' + 'F / Fmax')
plt.xlim([0.003, 0.08])
plt.ylabel("Normal theoretical quantile")
plt.legend(loc="lower right")

# annotation
for k, v in enumerate(dfof_ary):
    for index, row in dfof_ary[k].iterrows():
        ax.annotate(row['slot_num'],xy=(row['x'],row['y']),size=1, fontsize=2, color='black')

# plt.show()
plt.savefig("../images/anotation/p655.pdf")
plt.close()

# [参考]最小二乗法

In [17]:
def method_of_least_squares(x, y):
    """(docstring)
    """
    # 最小二乗法の数式:
    # y = bx + a (当記事で参考にした、東京大学出版会の『統計学入門』ではこの形)。
    # b = Σxiyi - nxbarybar / Σxi^2 - nxbar^2
    # a = ybar - bxbar

    mean_x = np.mean(x)
    mean_y = np.mean(y)
    n = len(x)

    b = (np.sum(x*y) - (n*mean_x*mean_y)) / (np.sum(x**2) - (n*mean_x**2))

    a = mean_y - b*mean_x

    return b, a